<a href="https://colab.research.google.com/github/diputs03/AI-Studies/blob/main/Creating_network/dymamic_architect_rebuilt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
#@title Aiming a Dynaimic Graph-structured NeuronNetwork
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
from collections import deque
import multiprocessing as mp

In [67]:
"""
Activation function in this case in \tanh, thus
\dfrac{d\tanh(x)}{dx}=1-\tanh^2(x)
however, for other activation funtions
\dfrac{d\sigma(x)}{dx}=\sigma(x)\cdot\left\big(1-\sigma(x)\right\big)
\dfrac{d\mathop{\mathrm{ReLu}}(x)}{dx}=\begin{cases}1&x\ge0\\0&\text{else}\end{cases}
Loss is the Euclidean loss
\dfrac{d\L}
"""
class Model:
  class Neuron:
    def __init__(self, name, prev, next):
      self.name, self.prev, self.next = name, prev, next
      self.bias = np.random.uniform(-0.1, 0.1)

  def __init__(self, input_size, output_size):
    self.Input_layer = [Model.Neuron(f"input{i}", [], []) for i in range(input_size)]
    self.Output_layer = [Model.Neuron(f"output{o}", [], []) for o in range(output_size)]
    for i in range(input_size): self.Input_layer[i].next = self.Output_layer
    for o in range(output_size): self.Output_layer[o].prev = self.Input_layer
    self.weight = {}
    self.weight_grad_sum, self.weight_grad_sqr = {}, {}
    for u in self.Input_layer:
      for v in self.Output_layer:
        self.weight[(u,v)] = np.random.uniform(-0.1, 0.1)
        self.weight_grad_sum[(u,v)], self.weight_grad_sqr[(u,v)] = 0, 0
    self.all_neurons = self.Input_layer + self.Output_layer

    self.bias_grad_sum, self.bias_grad_sqr = {}, {}
    for p in self.all_neurons:
      self.bias_grad_sum[p], self.bias_grad_sqr[p] = 0, 0

  def forward(self, X, batch_size):
    assert X.shape == (batch_size,len(self.Input_layer))
    a = {q: np.zeros(batch_size) for q in self.all_neurons}

    for i, n in enumerate(self.Input_layer):
      a[n] = X[:, i]

    q = deque()
    for i in self.Input_layer:
      q.append(i)

    cnt = {q: 0 for q in self.all_neurons}

    while len(q) != 0:
      c = q.popleft()
      a[c] = np.tanh(a[c] + c.bias)
      for n in c.next:
        a[n] = a[n] + a[c] * self.weight[(c,n)]
        cnt[n] += 1
        if cnt[n] == len(n.prev):
          q.append(n)
    return a

  def evaluate(self, X):
    a = self.forward(X, len(X))
    return np.array([a[o] for o in self.Output_layer]).T

  def backward(self, X, Y, batch_size, learning_rate, decay_sum, decay_sqr):
    assert X.shape == (batch_size,len(self.Input_layer))
    assert Y.shape == (batch_size,len(self.Output_layer))
    a = self.forward(X, batch_size)

    delta_b, delta_w = {}, {}

    par_a = {q: np.zeros(batch_size) for q in self.all_neurons}
    for o, n in enumerate(self.Output_layer):
      par_a[n] = 2 * (a[n] - Y[:, o])

    q = deque()
    for o in self.Output_layer:
      q.append(o)

    cnt = {q: 0 for q in self.all_neurons}

    while len(q) != 0:
      c = q.popleft()
      par_b = par_a[c] * (1-a[c]**2)

      grad_bias = par_b
      self.bias_grad_sum[c] = \
       (1-decay_sum)*np.sum(grad_bias)/batch_size + decay_sum*self.bias_grad_sum[c]
      self.bias_grad_sqr[c] = \
       (1-decay_sqr)*np.sum(grad_bias**2)/batch_size + decay_sqr*self.bias_grad_sqr[c]
      delta_b[c] = \
       -learning_rate * self.bias_grad_sum[c] / (self.bias_grad_sqr[c]**(1/2)+1)

      for p in c.prev:
        par_a[p] += par_a[c] * (1-a[c]**2) * self.weight[(p,c)]
        grad_weight = par_a[c] * (1-a[c]**2) * a[p]
        self.weight_grad_sum[(p,c)] = \
         (1-decay_sum)*np.sum(grad_weight)/batch_size + decay_sum*self.weight_grad_sum[(p,c)]
        self.weight_grad_sqr[(p,c)] = \
         (1-decay_sqr)*np.sum(grad_weight**2)/batch_size + decay_sqr*self.weight_grad_sqr[(p,c)]
        delta_w[(p,c)] = \
         -learning_rate * self.weight_grad_sum[(p,c)] / (self.weight_grad_sqr[(p,c)]**(1/2)+1)

        cnt[p] += 1
        if cnt[p] == len(p.next):
          q.append(p)

    return delta_w, delta_b

  def update(self, X, Y, batch_size, learning_rate, decay_sum=0.9, decay_sqr=0.9):
    delta_w, delta_b = \
     self.backward(X, Y, batch_size, learning_rate, decay_sum, decay_sqr)
    for (u,v), w in self.weight.items():
        self.weight[(u,v)] += delta_w[(u,v)]
    for p in self.all_neurons:
        p.bias += delta_b[p]

  def addLayer(self, mid_size, UP, DOWN):
    Mid_layer = [Model.Neuron(f"mid{o}", [], []) for o in range(mid_size)]
    for m, mid in enumerate(Mid_layer):
      self.bias_grad_sum[mid], self.bias_grad_sqr[mid] = 0, 0
      Mid_layer[m].prev = UP
      for u in UP:
        self.weight[(u,mid)] = np.random.uniform(-0.1, 0.1)
        self.weight_grad_sum[(u,mid)] = 0
        self.weight_grad_sqr[(u,mid)] = 0
      Mid_layer[m].next = DOWN
      for v in DOWN:
        self.weight[(mid,v)] = np.random.uniform(-0.1, 0.1)
        self.weight_grad_sum[(mid,v)] = 0
        self.weight_grad_sqr[(mid,v)] = 0
    for u in UP:
      u.next = Mid_layer
    for v in DOWN:
      v.prev = Mid_layer
    for u in UP:
      for v in DOWN:
        self.weight.pop((u, v))
        self.weight_grad_sum.pop((u, v))
        self.weight_grad_sqr.pop((u, v))
    self.all_neurons += Mid_layer
    return Mid_layer

  def train(self, X, Y, batch_size, epochs, learning_rate):
    l = len(X)
    for epoch in range(epochs):
      data=[(X[_], Y[_]) for _ in range(len(X))]
      random.shuffle(data)
      for _ in range(len(X)):
        X[_],Y[_]=data[_]
      loss = 0
      for batch in range(int(l / batch_size)):
        L, R = batch * batch_size, (batch + 1) * batch_size
        x_train, y_train = X[L:R], Y[L:R]
        self.update(x_train, y_train, batch_size, learning_rate)
        output = self.evaluate(x_train)
        loss += np.sum(((y_train - output) ** 2), axis=(0,1))
      loss = ((loss) ** 0.5) / (int(l / batch_size) * batch_size)
      print(f"Epoch {epoch}/{epochs}, Loss:{loss}")

In [68]:
X=np.array([[0,0],[0,1],[1,0],[1,1]])
Y=np.array([[0],[1],[1],[0]])
mod=Model(2, 1)
mid1=mod.addLayer(4, mod.Input_layer, mod.Output_layer)
mod.addLayer(4, mid1, mod.Output_layer)
mod.evaluate(X)

array([[0.08374906],
       [0.08421259],
       [0.08324875],
       [0.08371121]])

In [69]:
mod.train(X, Y, 4, 500, 0.1)

Epoch 0/500, Loss:0.3235096792489062
Epoch 1/500, Loss:0.38940287036996685
Epoch 2/500, Loss:0.31437984709720385
Epoch 3/500, Loss:0.37420955982640985
Epoch 4/500, Loss:0.30187930392481743
Epoch 5/500, Loss:0.09016971223149087
Epoch 6/500, Loss:0.09752100802246183
Epoch 7/500, Loss:0.10313059732982595
Epoch 8/500, Loss:0.10708290636532275
Epoch 9/500, Loss:0.10947287155667995
Epoch 10/500, Loss:0.11040248254257586
Epoch 11/500, Loss:0.10997822099739735
Epoch 12/500, Loss:0.1083093039855433
Epoch 13/500, Loss:0.10550658606051896
Epoch 14/500, Loss:0.10168195218402701
Epoch 15/500, Loss:0.09694803460060183
Epoch 16/500, Loss:0.09141810236906037
Epoch 17/500, Loss:0.0852059954951964
Epoch 18/500, Loss:0.07842600189312043
Epoch 19/500, Loss:0.07119260197877514
Epoch 20/500, Loss:0.06362003116991631
Epoch 21/500, Loss:0.055821634331698566
Epoch 22/500, Loss:0.047909008021575925
Epoch 23/500, Loss:0.03999094606892238
Epoch 24/500, Loss:0.03217222130418856
Epoch 25/500, Loss:0.024552250747976

In [70]:
print(X, Y)
mod.evaluate(X)

[[1 1]
 [1 1]
 [1 1]
 [1 1]] [[0]
 [0]
 [0]
 [0]]


array([[-1.83914516e-12],
       [-1.83914516e-12],
       [-1.83914516e-12],
       [-1.83914516e-12]])

In [71]:
import tensorflow as tf
tf.random.set_seed(42)
# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Flatten images to 1D vector of 784 features (28*28)
x_train = x_train.reshape(-1, 784).astype('float32') / 255.0
x_test = x_test.reshape(-1, 784).astype('float32') / 255.0

# One-hot encode the labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

def test(model, X, Y, batch_size):
  k = int(len(X)/batch_size)
  for i in range(k):
    Y_hat=model.evaluate(X[i*batch_size:(i+1)*batch_size])
    wrong=0
    for j in range(batch_size):
      max1,max2,id1,id2=-999,-999,-1,-1
      for l in range(10):
        if max1 < Y_hat[j][l]:
          max1,id1=Y_hat[j][l],l
        if max2 < Y[i*batch_size+j][l]:
          max2,id2=Y[i*batch_size+j][l],l
      if id1 != id2: wrong+=1
    print(f"batch: {i}, accuracy: {(batch_size-wrong)/batch_size*100}%")
mod2 = Model(784, 10)
mod2.addLayer(32, mod2.Input_layer, mod2.Output_layer)

In [76]:
print(len(mod2.Input_layer))
mod2.train(x_train, y_train, 2048, 5, 0.01)
"""
for i in range(1,9):
  plt.subplot(330+i)
  plt.imshow(x_test[i].reshape(28, 28), cmap=plt.get_cmap('gray'))
print(mod2.evaluate(np.array([x_test[5]])))
print(y_test[5])
"""
test(mod2, x_test, y_test, 500)

784
Epoch 0/5, Loss:0.0032852750452220065
Epoch 1/5, Loss:0.0032457336255522916
Epoch 2/5, Loss:0.0032104943934628083
Epoch 3/5, Loss:0.003181738407090361
Epoch 4/5, Loss:0.003154146300852825
batch: 0, accuracy: 71.6%
batch: 1, accuracy: 70.6%
batch: 2, accuracy: 65.8%
batch: 3, accuracy: 69.39999999999999%
batch: 4, accuracy: 67.80000000000001%
batch: 5, accuracy: 69.8%
batch: 6, accuracy: 74.2%
batch: 7, accuracy: 67.4%
batch: 8, accuracy: 66.4%
batch: 9, accuracy: 70.39999999999999%
batch: 10, accuracy: 80.0%
batch: 11, accuracy: 76.4%
batch: 12, accuracy: 84.2%
batch: 13, accuracy: 73.4%
batch: 14, accuracy: 78.2%
batch: 15, accuracy: 78.0%
batch: 16, accuracy: 77.60000000000001%
batch: 17, accuracy: 88.0%
batch: 18, accuracy: 78.0%
batch: 19, accuracy: 68.8%


In [77]:
test(mod2, x_test, y_test, 5000)

batch: 0, accuracy: 69.34%
batch: 1, accuracy: 78.25999999999999%
